In [16]:
import pandas as pd
import numpy as np
import csv as csv
import math
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from sklearn.cross_validation import train_test_split
import time
from datetime import date
import datetime
%matplotlib inline

def saveFileForSubmission(predicted_lables,custonFileName='submission.csv',customHeader=''):
    result = np.c_[predicted_lables]

    np.savetxt(custonFileName, 
           result.astype(int), 
           delimiter=',', 
           header = customHeader, 
           comments = '', 
           fmt='%u')

In [2]:
train_df = pd.read_csv('training_dataset.csv/training_dataset.csv', header=0)  
train_df.shape

(486048, 54)

In [3]:
def preProcessData(dataframe,train=True):
    mailTypes = ['mail_type_1', 'mail_type_2', 'mail_type_3', 'mail_type_4']
    mailCategories = ['mail_category_1', 'mail_category_10', 'mail_category_11',
                    'mail_category_12', 'mail_category_13', 'mail_category_14',
                    'mail_category_15', 'mail_category_16', 'mail_category_17',
                    'mail_category_18', 'mail_category_2', 'mail_category_3',
                    'mail_category_4', 'mail_category_5', 'mail_category_6',
                    'mail_category_7', 'mail_category_8', 'mail_category_9']
    mail_idRange = range(0,11)
    user_idRange = range(0,11)
    
    if(train):
        dataframe = dataframe.reindex_axis(['opened'] + list([a for a in dataframe.columns if a != 'opened']), axis=1)
        dataframe['opened'] = dataframe.clicked.map( {False: 0, True: 1} ).astype(int)
        dataframe = dataframe.drop(['clicked','unsubscribed','open_time','click_time','unsubscribe_time'], axis=1) 
        
    dataframe.loc[ (dataframe.last_online.notnull()),'last_online']=dataframe['last_online'].dropna().map(lambda x:(datetime.datetime.today()-datetime.datetime.fromtimestamp(x)).days)
    mean_last_online_days = dataframe.last_online.mean()
    dataframe.loc[ (dataframe.last_online.isnull()), 'last_online'] = mean_last_online_days
    
    mode_mail_type = dataframe.mail_type.dropna().mode().values
    dataframe.loc[ (dataframe.mail_type.isnull()), 'mail_type'] = mode_mail_type
    dataFrameMailTypesDiff = np.setdiff1d(mailTypes, np.unique(dataframe.mail_type)) 
    dummiesMail_type =  pd.get_dummies(dataframe.mail_type,prefix='col')
    dataframe = pd.concat([dataframe, dummiesMail_type], axis=1)
    for mailType in map(lambda x:"col_"+x,dataFrameMailTypesDiff):
        dataframe = pd.concat([dataframe,pd.DataFrame({mailType: np.zeros(dataframe.shape[0])})],axis=1)
    
    mode_mail_category = dataframe.mail_category.dropna().mode().values
    dataframe.loc[ (dataframe.mail_category.isnull()), 'mail_category'] = mode_mail_category
    dataFrameMailCategoryDiff = np.setdiff1d(mailCategories, np.unique(dataframe.mail_category)) 
    dummiesMail_category =  pd.get_dummies(dataframe.mail_category,prefix='col')
    dataframe = pd.concat([dataframe, dummiesMail_category], axis=1)
    for mailCategoryItem in map(lambda x:"col_"+x,dataFrameMailCategoryDiff):
        dataframe = pd.concat([dataframe,pd.DataFrame({mailCategoryItem: np.zeros(dataframe.shape[0])})],axis=1)
    
    dataframe['hacker_confirmation'] = dataframe.hacker_confirmation.map( {False: 0, True: 1} ).astype(int)
    
    dataTemp  = dataframe['mail_id'].value_counts()
    hist, edges = np.histogram(dataTemp, bins=[ 1.00000000e+00,2.62150000e+03,5.24200000e+03,
                                               7.86250000e+03,1.04830000e+04,1.31035000e+04,
                                               1.57240000e+04,1.83445000e+04,2.09650000e+04,
                                               2.35855000e+04,2.62060000e+04])
    dataTempDict = dataTemp.to_dict()
    dataframe['mail_id'] = dataframe['mail_id'].map(lambda x: [i for i,v in enumerate(edges) if v<=dataTempDict[x]][-1] )
    dummiesMail_id =  pd.get_dummies(dataframe.mail_id,prefix='mail_id')
    dataframe = pd.concat([dataframe, dummiesMail_id], axis=1)
    
    dataFrameMailIdDiff = np.setdiff1d(mail_idRange, np.unique(dataframe.mail_id)) 
    
    for mailIdItem in map(lambda x:"mail_id_"+str(x),dataFrameMailIdDiff):
        dataframe = pd.concat([dataframe,pd.DataFrame({mailIdItem: np.zeros(dataframe.shape[0])})],axis=1)
    
    dataTemp  = dataframe['user_id'].value_counts()
    hist, edges = np.histogram(dataTemp, bins=[1.,11.5,22.,32.5,43.,53.5,64.,74.5,85.,95.5,106.])
    dataTempDict = dataTemp.to_dict()
    dataframe['user_id'] = dataframe['user_id'].map(lambda x: [i for i,v in enumerate(edges) if v<=dataTempDict[x]][-1] )
    dummiesUser_id =  pd.get_dummies(dataframe.user_id,prefix='user_id')
    dataframe = pd.concat([dataframe, dummiesUser_id], axis=1)
    
    dataFrameUserIdDiff = np.setdiff1d(user_idRange, np.unique(dataframe.user_id)) 
    
    for userIdItem in map(lambda x:"user_id_"+str(x),dataFrameUserIdDiff):
        dataframe = pd.concat([dataframe,pd.DataFrame({userIdItem: np.zeros(dataframe.shape[0])})],axis=1)
    
    dataframe = dataframe.drop(['user_id','mail_id','hacker_created_at','sent_time' ,'hacker_timezone','mail_category','mail_type'], axis=1) 
    
    return dataframe
    

In [4]:
train_df = preProcessData(train_df)

In [5]:
train_df.head()

opened  last_online  contest_login_count  contest_login_count_1_days  \
0       0        152.0                    1                           0   
1       0        132.0                    3                           0   
2       0        107.0                    3                           0   
3       0        114.0                    3                           0   
4       0        132.0                    5                           0   

   contest_login_count_30_days  contest_login_count_365_days  \
0                            0                             1   
1                            1                             3   
2                            0                             3   
3                            0                             3   
4                            0                             5   

   contest_login_count_7_days  contest_participation_count  \
0                           0                            1   
1                           0                            3   
2                           0                            3   
3                           0                            3   
4                           0                           13   

   contest_participation_count_1_days  contest_participation_count_30_days  \
0                                   0                                    0   
1                                   0                                    1   
2                                   0                                    0   
3                                   0                                    0   
4                                   0                                    0   

     ...      user_id_1  user_id_2  user_id_3  user_id_4  user_id_5  \
0    ...            1.0        0.0        0.0        0.0        0.0   
1    ...            0.0        1.0        0.0        0.0        0.0   
2    ...            0.0        0.0        0.0        0.0        0.0   
3    ...            0.0        1.0        0.0        0.0        0.0   
4    ...            0.0        0.0        0.0        1.0        0.0   

   user_id_6  user_id_7  user_id_8  user_id_10  user_id_9  
0        0.0        0.0        0.0         0.0        0.0  
1        0.0        0.0        0.0         0.0        0.0  
2        0.0        0.0        0.0         0.0        0.0  
3        0.0        0.0        0.0         0.0        0.0  
4        0.0        0.0        0.0         0.0        0.0  

[5 rows x 86 columns]

In [6]:
train_df.columns

Index([u'opened', u'last_online', u'contest_login_count',
       u'contest_login_count_1_days', u'contest_login_count_30_days',
       u'contest_login_count_365_days', u'contest_login_count_7_days',
       u'contest_participation_count', u'contest_participation_count_1_days',
       u'contest_participation_count_30_days',
       u'contest_participation_count_365_days',
       u'contest_participation_count_7_days', u'forum_comments_count',
       u'forum_count', u'forum_expert_count', u'forum_questions_count',
       u'hacker_confirmation', u'ipn_count', u'ipn_count_1_days',
       u'ipn_count_30_days', u'ipn_count_365_days', u'ipn_count_7_days',
       u'ipn_read', u'ipn_read_1_days', u'ipn_read_30_days',
       u'ipn_read_365_days', u'ipn_read_7_days', u'submissions_count',
       u'submissions_count_1_days', u'submissions_count_30_days',
       u'submissions_count_365_days', u'submissions_count_7_days',
       u'submissions_count_contest', u'submissions_count_contest_1_days',
       

In [7]:
from sklearn import linear_model
from sklearn import tree
from sklearn import svm


train_data = train_df.values
x_train, x_test, y_train, y_test = train_test_split(train_data[0::,1::], train_data[0::,0], 
                            test_size = 0.2, random_state = 0) # Split training/test.

#hipotese = linear_model.LogisticRegression(C=1e5)
hipotese = tree.DecisionTreeClassifier(random_state=0)
#hipotese = svm.SVC()
hipotese.fit(x_train, y_train )


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [8]:
y_true, y_pred = y_test, hipotese.predict(x_test) # Get our predictions
print(classification_report(y_true, y_pred)) # Classification on each digit


             precision    recall  f1-score   support

        0.0       0.95      0.95      0.95     91858
        1.0       0.16      0.17      0.16      5352

avg / total       0.91      0.90      0.91     97210



In [9]:
test_df = pd.read_csv('test_dataset.csv/test_dataset.csv', header=0)  
test_df.shape

user_id  \
0  3EwevzgIuZX9hq2zhf1Mz2C/xOd5Rf+B793bBOTSkRA=   
1  blQUjF8SGLYtv1K236MnflXw0X6ie8QJHZy0SDW5Llg=   
2  OEfFUcsTAGInCfsHuLZuIgdSNtuNsg8EdfN98VUZVTs=   
3  4L/sXU2l7/1DyWK5BdEkmex0yBiyHaAW9yGPHMtrDOw=   
4  0OfcrqLw8m4qYC0CLqX0DvmzBV/OQZNa9GNXQKHZdK4=   

                                        mail_id    mail_category    mail_type  \
0  BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=  mail_category_1  mail_type_1   
1  EHNBRbi6i9KO6cMHsuDPFjZVp2cY3RH+BiOKwPwzLQs=  mail_category_1  mail_type_1   
2  UqjHLTfci2jErniqKM2JHBn8DurSys3+UlZOpSa32t4=  mail_category_3  mail_type_1   
3  BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=  mail_category_1  mail_type_1   
4  K0y/NW59TJkYc5y0HUwDeAXrewYT0JQlkcozz0s2V5Q=  mail_category_4  mail_type_1   

    sent_time   last_online  hacker_created_at  hacker_timezone  \
0  1467708425  1.467620e+09         1433145409          18000.0   
1  1466570440  1.466483e+09         1433734262          18000.0   
2  1463671887  1.463411e+09         1433045937          18000.0   
3  1467719224  1.467632e+09         1432109057          18000.0   
4  1467723250  1.467116e+09         1432012189          18000.0   

   contest_login_count  contest_login_count_1_days  \
0                    1                           0   
1                    3                           0   
2                    3                           0   
3                    2                           0   
4                    1                           0   

                ...                 submissions_count_contest  \
0               ...                                         0   
1               ...                                         7   
2               ...                                         0   
3               ...                                        41   
4               ...                                         0   

   submissions_count_contest_1_days  submissions_count_contest_30_days  \
0                                 0                                  0   
1                                 0                                  0   
2                                 0                                  0   
3                                 0                                  0   
4                                 0                                  0   

   submissions_count_contest_365_days  submissions_count_contest_7_days  \
0                                   0                                 0   
1                                   3                                 0   
2                                   0                                 0   
3                                  41                                 0   
4                                   0                                 0   

   submissions_count_master  submissions_count_master_1_days  \
0                        21                                0   
1                        28                                0   
2                        16                                0   
3                         1                                0   
4                        42                                0   

   submissions_count_master_30_days  submissions_count_master_365_days  \
0                                 0                                 21   
1                                 0                                 28   
2                                 3                                 16   
3                                 0                                  1   
4                                 1                                 29   

   submissions_count_master_7_days  
0                                0  
1                                0  
2                                0  
3                                0  
4                                0  

[5 rows x 48 columns]

In [10]:
test_df = preProcessData(test_df,False)
test_df.shape

last_online  contest_login_count  contest_login_count_1_days  \
0         58.0                    1                           0   
1         71.0                    3                           0   
2        107.0                    3                           0   
3         58.0                    2                           0   
4         64.0                    1                           0   

   contest_login_count_30_days  contest_login_count_365_days  \
0                            0                             0   
1                            0                             1   
2                            0                             3   
3                            0                             1   
4                            0                             0   

   contest_login_count_7_days  contest_participation_count  \
0                           0                            1   
1                           0                            4   
2                           0                            3   
3                           0                            2   
4                           0                            1   

   contest_participation_count_1_days  contest_participation_count_30_days  \
0                                   0                                    0   
1                                   0                                    0   
2                                   0                                    0   
3                                   0                                    0   
4                                   0                                    0   

   contest_participation_count_365_days     ...      user_id_1  user_id_2  \
0                                     0     ...            0.0        0.0   
1                                     2     ...            0.0        0.0   
2                                     3     ...            0.0        0.0   
3                                     1     ...            1.0        0.0   
4                                     0     ...            1.0        0.0   

   user_id_3  user_id_4  user_id_5  user_id_6  user_id_7  user_id_8  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0   
2        0.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0   

   user_id_9  user_id_10  
0        0.0         0.0  
1        0.0         0.0  
2        0.0         0.0  
3        0.0         0.0  
4        0.0         0.0  

[5 rows x 85 columns]

In [12]:
test_data = test_df.values

y_pred = hipotese.predict(test_data).astype(int)


In [17]:
saveFileForSubmission(y_pred,'submission.csv',)